In [8]:
import glob
import os
import pandas as pd
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go

from datetime import datetime
from openpyxl import Workbook
from openpyxl.drawing.image import Image
from plotly.subplots import make_subplots
from pptx.util import Cm
from pptx import Presentation

In [9]:
pio.templates.default = "simple_white"

In [10]:
list_of_files = glob.glob('sources\*') # liste tous les fichiers du dossier

list_of_files = sorted(list_of_files, key=os.path.getmtime) #sort via la date de création (le dernier le plus récent)

In [11]:
demandes = pd.read_excel(list_of_files[-2]) #génére un df à partir du pelnutième fichier du dossier

In [12]:
demandes2 = pd.read_excel(list_of_files[-1])#génére un df à parti du fichier le plus récent du dossier

In [13]:
wb=Workbook() # créé un workbook pour export excel

In [14]:
ppt = Presentation("template.pptx") #ouvre une présentation powerpoint


In [15]:
first_slide = ppt.slides[0] # première slide
  

In [16]:
subtitle = first_slide.placeholders[1] #sous_titre
subtitle.text = str(demandes2["Date d'émission"].dt.date.max())

In [17]:
pres = "presentation-{}.pptx".format(str(demandes2["Date d'émission"].dt.date.max()))
ppt.save(pres)

# fonctions

In [18]:
def isEncours( df):
    '''créé une nouvelle colonne (En cours) selon le Statut'''
    for i in range(len(df)):
        if df.loc[i, "Statut"]=="Résolu":
            df.loc[i,"En cours"]="Non"
        elif df.loc[i, "Statut"]=="Clôturé":
            df.loc[i,"En cours"]="Non"
        elif df.loc[i, "Statut"]=="Refusé Operation":
            df.loc[i,"En cours"]="Non"
        else:
            df.loc[i,"En cours"]='Oui'


In [19]:
def compare_stocks_demandes(df, df2):
    ''' Génére un graphique en barre comparant les stocks d événements entre deux périodes puis l'exporte dans un fichier
    Excel et dans une présentation prowerpoint spécifiés en amont
    '''
    T0 = str(df["Date d'émission"].dt.date.max())
    T1 = str(df2["Date d'émission"].dt.date.max())
    dft = pd.DataFrame.from_dict({T0: [df['En cours'].value_counts()[0], df['En cours'].value_counts()[1]], 
                                 T1: [df2['En cours'].value_counts()[0],df2['En cours'].value_counts()[1]]}, 
                                orient='index', columns=["Non", "Oui"]
                            )

    fig = go.Figure()
    fig.add_trace( go.Bar(y=[T0,T1], x=dft["Oui"],
                      marker_color= "#F58518",
                      orientation='h',
                      name="En cours",
                      text=dft["Oui"], 
                      
    ))
    fig.add_trace(go.Bar(y=[T0,T1], x=dft["Non"],
                     marker_color= 'dimgrey',
                     orientation='h',
                     name="Fini",
                     text= dft["Non"],
                    
    ))
    fig.add_trace(go.Scatter(y=[T0,T1], x = list(dft.sum(axis=1)), 
                         mode = "lines+markers+text",text=list(dft.sum(axis=1)),
                         textfont=dict( size=15, color="#B22222"),
                         textposition="middle right",
                         marker_color = "#B22222",showlegend = False
                        ))
    fig.update_layout( width=900,
                    barmode="group",bargap=0.1,
                     title="Nombre de demandes", 
                     legend=dict(xanchor="right", yanchor="top", orientation="h"))
    fig.update_xaxes(visible=False,range=[0,dft.sum(axis=1).max()+120])
    fig.update_yaxes(type='category' )

    fig.write_image("fig.png", engine="kaleido") # nécessite l'intallation de kaleido
    
    # ajoute l'image du graph à un fichier excel
    sheet1 = wb.create_sheet("stock",0)
    active=wb["stock"]
    active.add_image(Image("fig.png"),"A1")
    wb.save("suivi_de_demande_"+T1+".xlsx")
    
    # ajoute l'image du graph à une présentation powerpoint
    prs = Presentation(pres) # on charge la présentation avec la première slide modifiée
    slide2_layout = prs.slide_layouts[6] # layout d'une page blanche (cf menu ajouter une slide de powerpoint)
    slide2= prs.slides.add_slide(slide2_layout) # ajoute une slide à la présentation
    left=Cm(1)
    top=Cm(0.6)
    pic = slide2.shapes.add_picture("fig.png", left, top)
    
    prs.save(pres)
    

# Sum up

In [20]:
isEncours(demandes)
isEncours(demandes2)

In [21]:
compare_stocks_demandes(demandes, demandes2)